In [7]:
from bs4 import BeautifulSoup
import pymysql
import requests
# import library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set_palette('Set3')

import missingno as msno
import warnings
warnings.filterwarnings('ignore')

import scipy.stats

plt.rcParams["font.family"] = "NanumBarunGothic"

from category_encoders.target_encoder import TargetEncoder

In [8]:
# data load
# pip install openpyxl
path = "C:/프로젝트/k-d/2-Project/주제/data"
# main = pd.read_excel(path + '/국립중앙도서관 대출 데이터/NL_CO_LOAN_PUB_202311-7.xlsx')
Top_Lib = pd.read_excel(path + '/크롤링select데이터/제목_ISBN중복제거.xlsx')



num_rows, num_columns = Top_Lib.shape

# 결과 출력
print(f"데이터 수: {num_rows}")

plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

데이터 수: 10100


In [9]:
# Test 실행
# #headers = {
#   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
#}

#for index, count in Top_Lib.iterrows():
#    response = requests.get("https://www.yes24.com/Product/Search?domain=ALL&query={0}".format(count['ISBN13번호']), headers=headers)
#    URL = ("https://www.yes24.com/Product/Search?domain=ALL&query={0}".format(count['ISBN13번호']))
#    print("URL 확인", URL)

In [56]:
conn = pymysql.connect(host='127.0.0.1',
                       user='root',
                       password='1234',
                       db='Test_one',
                       charset='utf8')
cursor = conn.cursor()

headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for index, count in Top_Lib.iterrows():
    response = requests.get("https://search.kyobobook.co.kr/search?keyword={0}&gbCode=TOT&target=total".format(count['ISBN13번호']), headers=headers)
    URL = ("https://search.kyobobook.co.kr/search?keyword={0}&gbCode=TOT&target=total".format(count['ISBN13번호']))
    # print("URL 확인", URL)
        
    html = response.content.decode('utf-8','replace')
    
    soup = BeautifulSoup(html, 'html.parser',from_encoding='cp949')

    my_st = soup.select(
        '#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box'
    )
    
    # print(my_st)  
    # #yesSchList > li > div > div.item_info > div.info_row.info_price > strong

    
    for stock_name in my_st:
        try:
            price_elements = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_price > span.price > span.val")

            if price_elements and price_elements[0]:
                Price = price_elements[0].text
            else:
                Price = "0원"

        except IndexError as e:
            print("Price 처리 중 에러:", e)
            Price = "0원"

        try:
            Vente_exposant = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_bottom > a > span.review_klover_box > span.review_klover_text.font_size_xxs")

            if Vente_exposant and Vente_exposant[0]:
                Vente_exposant_text = Vente_exposant[0].text
            else:
                Vente_exposant_text = "0"

        except IndexError as e:
            print("Vente_exposant 처리 중 에러:", e)
            Vente_exposant_text = "0"

        try:
            review_desc = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_bottom > a > span.review_klover_box > span.review_desc")
       
            if review_desc and review_desc[0]:
                review_desc_text = review_desc[0].text
            else:
                review_desc_text = "0"

        except IndexError as e:
            print("review_desc 처리 중 에러:", e)
            review_desc_text = "0"
            
    sql = "INSERT INTO Test_one.Vente_exposant (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('{0}', {1}, {2});".format(str(count['ISBN13번호']), "'"+Price+"'", Vente_exposant, review_desc)
    print(sql)
            
#cursor.execute(sql)
#conn.commit()
# conn.close()

INSERT INTO Test_one.Vente_exposant (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('9791197522901', '16,200', [<span class="review_klover_text font_size_xxs">9.4</span>]);
INSERT INTO Test_one.Vente_exposant (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('9791197510601', '15,300', [<span class="review_klover_text font_size_xxs">9.89</span>]);
INSERT INTO Test_one.Vente_exposant (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('9791197488900', '18,000', [<span class="review_klover_text font_size_xxs">9.62</span>]);
INSERT INTO Test_one.Vente_exposant (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('9791197488207', '16,200', [<span class="review_klover_text font_size_xxs">9.54</span>]);
INSERT INTO Test_one.Vente_exposant (ISBN_THIRTEEN_NO, price, Vente_exposant, review_desc) VALUES('9791197472510', '16,200', [<span class="review_klover_text font_size_xxs">9.54</span>]);


KeyboardInterrupt: 